In [3]:
import pandas as pd
import statsmodels.api as sm

data = {
    'Sila_wiatru': ['silny', 'silny', 'brak', 'brak', 'slaby', 'slaby', 'brak', 'silny', 'brak', 'silny'],
    'Zachmurzenie': ['pochmurnie', 'pochmurnie', 'slonecznie', 'slonecznie', 'pochmurnie', 'slonecznie', 'pochmurnie', 'slonecznie', 'pochmurnie', 'pochmurnie'],
    'Temperatura': ['zimno', 'cieplo', 'cieplo', 'goraco', 'goraco', 'cieplo', 'zimno', 'zimno', 'goraco', 'cieplo'],
    'Zagrano': ['nie', 'nie', 'tak', 'nie', 'tak', 'tak', 'nie', 'tak', 'tak', 'tak']
}
df = pd.DataFrame(data)

# LABEL ENCODING
map_wiatr = {'brak': 0, 'slaby': 1, 'silny': 2}
map_chmury = {'pochmurnie': 0, 'slonecznie': 1}
map_temp = {'zimno': 0, 'cieplo': 1, 'goraco': 2}
map_zagrano = {'nie': 0, 'tak': 1}

df['Wiatr_X'] = df['Sila_wiatru'].map(map_wiatr)
df['Chmury_X'] = df['Zachmurzenie'].map(map_chmury)
df['Temp_X'] = df['Temperatura'].map(map_temp)
df['y'] = df['Zagrano'].map(map_zagrano)

X = df[['Wiatr_X', 'Chmury_X', 'Temp_X']]
X = sm.add_constant(X)
y = df['y']

model = sm.Logit(y, X).fit()
print(model.summary())

# Test - "ciepło (1), słonecznie (1), silny wiatr (2)"
# [const, Wiatr, Chmury, Temp]
nowe_dane = [1, 2, 1, 1]

prawdopodobienstwo = model.predict(nowe_dane)[0]
print(f"\nPredykcja dla (Silny, Słonecznie, Ciepło): {prawdopodobienstwo:.4f}")
print("Zagrano mecz:", "tak" if prawdopodobienstwo > 0.5 else "nie")

Optimization terminated successfully.
         Current function value: 0.583117
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                   10
Model:                          Logit   Df Residuals:                        6
Method:                           MLE   Df Model:                            3
Date:                Sun, 25 Jan 2026   Pseudo R-squ.:                  0.1336
Time:                        11:05:23   Log-Likelihood:                -5.8312
converged:                       True   LL-Null:                       -6.7301
Covariance Type:            nonrobust   LLR p-value:                    0.6154
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.8566      2.395     -0.775      0.438      -6.550       2.837
Wiatr_X        0.5986      0.

In [9]:
import pandas as pd
import statsmodels.api as sm
import importlib

klasyfikatory = importlib.import_module('Lab-3-Zadanie-1-Klasyfikatory')

dane_uczace = pd.DataFrame({
    'pieniądz':       ['nie', 'tak', 'nie', 'nie', 'tak', 'nie', 'nie', 'nie', 'nie', 'nie', 'tak', 'tak', 'nie'],
    'darmowy':        ['nie', 'tak', 'nie', 'tak', 'nie', 'tak', 'tak', 'nie', 'tak', 'nie', 'tak', 'nie', 'tak'],
    'bogaty':         ['tak', 'tak', 'nie', 'nie', 'nie', 'nie', 'nie', 'nie', 'nie', 'nie', 'tak', 'nie', 'tak'],
    'nieprzyzwoicie': ['nie', 'nie', 'nie', 'nie', 'nie', 'tak', 'tak', 'tak', 'nie', 'nie', 'nie', 'nie', 'nie'],
    'tajny':          ['tak', 'nie', 'nie', 'nie', 'nie', 'tak', 'nie', 'nie', 'nie', 'tak', 'tak', 'tak', 'nie'],
    'spam':           ['tak', 'tak', 'nie', 'tak', 'nie', 'tak', 'tak', 'tak', 'nie', 'nie', 'tak', 'tak', 'nie']
})

dane_testowe = pd.DataFrame({
    'pieniądz':       ['tak'],
    'darmowy':        ['nie'],
    'bogaty':         ['tak'],
    'nieprzyzwoicie': ['nie'],
    'tajny':          ['tak']
})

mapping = {'tak': 1, 'nie': 0}
df_logit = dane_uczace.replace(mapping)
X_test_logit = dane_testowe.replace(mapping)

X_logit = df_logit.drop('spam', axis=1)
y_logit = df_logit['spam']

X_logit = sm.add_constant(X_logit)
X_test_logit_const = sm.add_constant(X_test_logit, has_constant='add')

logit_model = sm.Logit(y_logit, X_logit).fit(disp=0)

print(logit_model.summary())

prob = logit_model.predict(X_test_logit_const)[0]
decyzja_logit = "tak" if prob > 0.5 else "nie"

print(f"Prawdopodobieństwo spamu: {prob:.4f}")
print(f"Klasyfikacja modelem logitowym: {decyzja_logit}")

print("\n\n\n")
X_train_knn = dane_uczace.drop('spam', axis=1)
y_train_knn = dane_uczace['spam']

k = 3
wynik_knn = klasyfikatory.knn(
    X_train_knn,
    y_train_knn,
    dane_testowe,
    k,
    klasyfikatory.manhattan_dist
)

klasyfikatory.print_result(dane_testowe, wynik_knn, "spam")

                           Logit Regression Results                           
Dep. Variable:                   spam   No. Observations:                   13
Model:                          Logit   Df Residuals:                        7
Method:                           MLE   Df Model:                            5
Date:                Sun, 25 Jan 2026   Pseudo R-squ.:                  0.6253
Time:                        11:28:22   Log-Likelihood:                -3.2454
converged:                      False   LL-Null:                       -8.6616
Covariance Type:            nonrobust   LLR p-value:                   0.05480
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
const            -60.7294   6.08e+06  -9.99e-06      1.000   -1.19e+07    1.19e+07
pieniądz          29.5660    1.6e+06   1.85e-05      1.000   -3.14e+06    3.14e+06
darmowy           59.8220   6.08e+06

/tmp/ipython-input-974558755.py:25: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_logit = dane_uczace.replace(mapping)
/tmp/ipython-input-974558755.py:26: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_test_logit = dane_testowe.replace(mapping)
/usr/local/lib/python3.12/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
